<a href="https://colab.research.google.com/github/ridhokurniawan-u/Sentiment-Model/blob/main/Sentiment_TrainingModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras_nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 23.1 MB/s eta 0:00:00


In [8]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')

# Load the IMDb dataset
max_features = 10000  # Only consider the top 10,000 words in the dataset
maxlen = 100  # Cut reviews after 100 words
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# Pad sequences to have consistent length for input to the model
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

# Build the model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=15, batch_size=32, validation_split=0.2)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_acc}')

# Save the model to Google Drive
model_save_path = '/content/gdrive/My Drive/Sentiment_Model.h5'
model.save(model_save_path)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Epoch 1/15
625/625 [==============================] - 22s 33ms/step - loss: 0.4329 - accuracy: 0.7934 - val_loss: 0.3478 - val_accuracy: 0.8520
Epoch 2/15
625/625 [==============================] - 6s 10ms/step - loss: 0.2555 - accuracy: 0.8976 - val_loss: 0.4726 - val_accuracy: 0.8316
Epoch 3/15
625/625 [==============================] - 6s 10ms/step - loss: 0.1727 - accuracy: 0.9354 - val_loss: 0.4495 - val_accuracy: 0.8422
Epoch 4/15
625/625 [==============================] - 5s 8ms/step - loss: 0.1181 - accuracy: 0.9572 - val_loss: 0.4627 - val_accuracy: 0.8362
Epoch 5/15
625/625 [==============================] - 6s 9ms/step - loss: 0.0858 - accuracy: 0.9700 - val_loss: 0.5717 - val_accuracy: 0.8352
Epoch 6/15
625/625 [==============================] - 5s 8ms/step - loss: 0.0648 - accuracy: 0.9790 - val_loss: 0.5423 - val_accuracy: 0.8298
Epoch 7/15
62

In [9]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')

# Load the IMDb dataset (for test data)
max_features = 10000  # Only consider the top 10,000 words in the dataset
maxlen = 100  # Cut reviews after 100 words
(_, _), (x_test, y_test) = imdb.load_data(num_words=max_features)

# Pad sequences to have consistent length for input to the model
x_test = pad_sequences(x_test, maxlen=maxlen)

# Load the saved model
model_save_path = '/content/gdrive/My Drive/Sentiment_Model.h5'
model = tf.keras.models.load_model(model_save_path)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_acc}')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
782/782 [==============================] - 3s 4ms/step - loss: 1.0054 - accuracy: 0.8262
Test Accuracy: 0.8262400031089783


In [11]:
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.datasets import imdb
from google.colab import drive

# Function to load the saved model
def load_model(model_path):
    model = tf.keras.models.load_model(model_path)
    return model

# Function to preprocess the text data
def preprocess_text(text, word_index, maxlen=100):
    tokens = text_to_word_sequence(text)
    tokens = [word_index.get(word, 0) for word in tokens]
    padded = pad_sequences([tokens], maxlen=maxlen)
    return padded

# Function to predict sentiment
def predict_sentiment(review, model, word_index):
    processed_review = preprocess_text(review, word_index)
    prediction = model.predict(processed_review)
    return prediction[0][0]

# Function to classify sentiment
def classify_sentiment(prediction):
    return "Positive" if prediction > 0.5 else "Negative"

# Function to process reviews and write results to a file
def analyze_reviews(json_path, model, word_index, output_path):
    with open(json_path, 'r') as file:
        data = json.load(file)

    with open(output_path, 'w') as file:
        for item in data:
            review = item['quotes']
            prediction = predict_sentiment(review, model, word_index)
            sentiment = classify_sentiment(prediction)
            file.write(f'Review: {review}\nSentiment: {sentiment} ({prediction * 100:.2f}% confidence)\n\n')

# Mount Google Drive to access the model and the JSON file
drive.mount('/content/gdrive')

# Paths
model_path = '/content/gdrive/MyDrive/Sentiment_Model.h5'
json_path = '/content/gdrive/MyDrive/Sentiment_Model/reformatted_quotes_AbsolutelyAnything.json'
output_path = '/content/gdrive/MyDrive/Sentiment_Model/sentiment_results_AbsolutelyAnything.txt'

# Load the model and IMDb word index
model = load_model(model_path)
word_index = imdb.get_word_index()

# Analyze the reviews
analyze_reviews(json_path, model, word_index, output_path)

print("Sentiment analysis complete. Results saved to sentiment_results.txt.")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
1/1 [==============================] - 0s 26ms/step
Sentiment analysis complete. Results saved to sentiment_results.txt.
